# Creación y cálculo de las etiquetas: B_Añadir_Etiquetas.ipynb
En este *notebook* cargamos el binario que contiene nuestro censo procesado por spacy-stanza. A continuación seguiremos un proceso denso, pero repetivo para determinar el año, parroquia y localidad asignada a cada persona del censo. Para cada característica se sigue el mismo método interactivo:
- Encontrar expresiones en el texto que puedan indicar la característica buscada. Por ejemplo, si encontramos la frase "parroquia de".
- A continuación recorremos una a una todas las coincidencias. Muchas veces, el usuario no tendrá que verificar una a una todas las coincidencias, porque se ha programado ya muchos casos para que la respuesta sea automática. En otros deberá intervenir, por ejemplo para determinar si un Carcedo es el Carcedo de Lago o el Carcedo de Lomes.
- Por último, asignamos a todas las personas indicadas dicha característica en una extensión de *Span* (ver documentación de SpaCy). Las caracterísiticas asignadas están programadas para encontrar información que se ha recogido en el *notebook* Z_Ayudas, por ejemplo, la geolocalización.

## Inicio
### Importamos las librerías a usar
Lo haremos siempre al comienzo de cada *notebook*.

In [1]:
import spacy
import spacy_stanza
from spacy import displacy
from spacy.tokens import  Span, DocBin
from spacy.matcher import Matcher, PhraseMatcher

# Ejecutamos nuestras funciones y ayudas creadas para la ocasión. Están en el notebook Z_Funciones.ipynb y Z_Ayudas.ipynb.
%run './Z_Funciones.ipynb'
%run './Z_Ayudas.ipynb'

### Especificamos la direcciones y configuraciones a usar
Esta sección es la que podemos personalizar o modificar a conveniencia según el archivo que queramos procesar, dónde lo queremos guardar, etc. Contiene los siguientes datos:
- `direccionbin`: es la dirección donde vamos a guardar el archivo procesado por spacy-stanza. Es un archivo binario.
- `nlpconfig`: es la variable que determina las características del pipeline de procesamiento de spacy-stanza. Por ejemplo, contiene el idioma, cuántas frases procesamos de una sola vez, etc.
- `direcciontxt`: es la dirección donde tenemos guardado el txt de estudio.
- `direcciónpueblos`: es la dirección donde donde se guarda el archivo csv con la información de las entradas y su geolocalización.

In [2]:
direccionbin = './Documentos/Padronbin'
direccionbin_tags = './Documentos/Padronbin_tags'
direcciontxt = './Documentos/PadronOCRPrueba.txt'
direcciónpueblos = './Documentos/GeolocLocalidadesPrueba.tsv'
# direccionbin = './Privado/PadronbinTOTAL'
# direccionbin_tags = './Privado/Padronbin_tagsTOTAL'
# direcciontxt = './Privado/PadronOCR.txt'
# direcciónpueblos = './Privado/GeolocLocalidadesTOTAL.tsv'
nlpconfig = {
    'name': 'es', # Language code for the language to build the Pipeline in
    'tokenize_batch_size': 32, # Enseguida vamos a cambiarlo por la cantidad máxima de palabras que alberga una frase en nuestro documento.
    'ner_batch_size': 32 # Enseguida vamos a cambiarlo por la cantidad máxima de palabras que alberga una frase en nuestro documento.
}

In [3]:
# Contamos las palabras y corregimos el diccionario nlpconfig
numero1 = cuenta_palabras_max(direcciontxt)
numero2 = int(numero1 * 1.25)

nlpconfig['ner_batch_size'] = numero2
nlpconfig['tokenize_batch_size'] = numero2

print("El número de palabras máximas es", str(numero1) + ".", "\nAñadiendo un 25%, tomaremos el número", str(numero2) + ".")

El número de palabras máximas es 33. 
Añadiendo un 25%, tomaremos el número 41.


### Cargamos el pipeline de procesamiento

Creamos los pipelines (las funciones que procesan nuestro archivo de análisis). `nlp1` es el pipeline de spacy-stanza y `nlp2` es el de spacy

In [4]:
nlp = spacy_stanza.load_pipeline(**nlpconfig)

2021-08-27 09:21:28 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

/home/carlos-eduardo/anaconda3/envs/PMLatest/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
2021-08-27 09:21:28 INFO: Use device: cpu
2021-08-27 09:21:28 INFO: Loading: tokenize
2021-08-27 09:21:28 INFO: Loading: mwt
2021-08-27 09:21:28 INFO: Loading: pos
2021-08-27 09:21:32 INFO: Loading: lemma
2021-08-27 09:21:32 INFO: Loading: depparse
2021-08-27 09:21:35 INFO: Loading: ner
2021-08-27 09:21:44 INFO: Done loading processo

### Revertimos la serializamos
La palabra serializar se refiere a guardar nuestro documento procesado `doc` a un archivo binario que se pueda alojar en un disco duro. Así no tenemos que procesar el documento de texto cada vez que abrimos *Jupyter Lab*. Cargamos el binario en la dirección `direccionbin`. Para ello usaremos la función `revertir_serializacion` definida en el archivo *funciones_Spacy.ipynb*.

In [5]:
doc = revertir_serializacion(direccionbin, nlp)

Nota: esta función requiere los siguientes módulo o funciones:
 import spacy
 from spacy.tokens import Doc
 from spacy.tokens import DocBin
 Si no los usa se producirá un error.


### Mirando las entidades nombradas que hemos resuelto
Vamos a generar un documento de texto que muestre cada frase señalando las EN resueltas. Para la lengua española, spacy-stanza resuelve tres tipos de entidad nombrada, a saber, personas (PER), lugar (LOC), organización (ORG) y miscelanea (MISC).


In [6]:
inicio = 0
final = 110
colors = {"PER": "#6EC5FF",
         "LOC": "#F16EFF",
         "ORG": "#FFA86E",
         "MISC": "#7DFF6E"}
options = {"ents": ["PER", "LOC", "ORG", "MISC"], "colors": colors}

displacy.render(doc[inicio:final].as_doc(), style="ent", jupyter=True, options=options)

## Obtención del año
### Aplicamos un patrón de búsqueda
Creamos un diccionario cuyos valores son todos los años en la expresión "Año xxxx" y cuyas claves son el identificador del token que lo contiene. 

In [7]:
# Inicializa el matcher con el vocabulario compartido
matcher = Matcher(nlp.vocab)

# Crea un patrón que encuentre "Año xxxx".
pattern = [{"LOWER": "año"}, {"IS_DIGIT": True}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("AÑO", [pattern])
matches = matcher(doc)

# Inicializamos el diccionario
años = {}

# Itera sobre los resultados para rellenar el diccionario `años`
for match_id, start, end in matches:
    años[start+1] = int(doc[start+1].text)

Si deseas ver cómo es este diccionario, descomenta y ejecuta el siguiente código

In [8]:
# años

### Asignamos a cada persona un año
Vamos a crear para ello una extensión para cada entidad denominada `tiempo`.

In [9]:
Span.set_extension("tiempo", default=False, force=True)

Ahora calculamos la etiqueta en cada entidad nombrada. Esto es una tarea tediosa porque hay que aplicar computacionalmente la regla: a cada entidad le corresponde el año indicado en el primer *token* por encima de ella misma y que se encuentra en el diccionario `años`. Veámoslo con un ejemplo del extracto de prueba:
- La persona "Bartolomé Fernández" comienza en el token 147 y termina en el 149. Constátelo ejecutanto `doc[147:149]`.
- El diccionario de `años` muestra que al token 7 le corresponde el año 1698. Al token 160 el 1773. Constátelo ejecutanto `años[7]` y `años[160]`.
- Está claro que a esta persona le corresponde el año 1698, del token 7. Programaremos el año que está justo encima.

In [10]:
# Creamos una lista con las claves del diccionario `años`. Y variables auxiliares.
años_claves = list(años.keys())
años_claves.sort()
años_len = len(años_claves)
aux0 = 0
aux1 = 1

# Ahora recorremos las entidades del documento y asignamos el año en cada entidad nombrada
for ent in doc.ents:
    if ent.label_ == "PER":
        # Calculamos el id del token que comienza la entidad:
        comienzo = ent.start
        
        if comienzo < años_claves[0]:
            continue
        else:
            if años_claves[aux0] < comienzo < años_claves[aux1]:
                ent._.tiempo = años.get(años_claves[aux0])
            elif comienzo > años_claves[aux0] and comienzo > años_claves[aux1]:
                aux0 = aux1
                if aux1 + 1 < años_len:
                    aux1 += 1
                    ent._.tiempo = años.get(años_claves[aux0])
                else:
                    años_claves.append(comienzo + 1000)
                    aux1 += 1
                    ent._.tiempo = años.get(años_claves[aux0])



### Veamos algunos ejemplos de lo que hemos conseguido

In [11]:
inicio = 130
final = 190

for ent in doc.ents:
    if ent.label_ == 'PER' and (inicio <= ent.start <= final):
         print(ent.text, '--- Año:', ent._.tiempo)

Simon Queipo --- Año: 1698
Bartolome Fernandez --- Año: 1698
Don Joseph Antonio de Herías --- Año: 1773
Juan García de Zibran --- Año: 1773


## Obtención de la parroquia
En el *notebook* `Z_Ayudas.ipynb` tenemos una lista con todas las entradas de parroquias del censo. Creamos un patrón de Phrase Matcher que reconozca cada una y la etiquete convenientemente. Vamos a usar la función `verificar_matcher` para grabar las etiquetas. Por ejemplo, en la frase "PARROQUIA DE ARANIEGO(PARAJAS)", se le asignará la etiqueta `Parajas`. 

IMPORTANTE: Todas las entradas de parroquias del documento txt deben estar en mayúsculas y en líneas aisladas.

### Aplicamos un patrón de búsqueda usando variables de apoyo
El *notebook* Z_Ayudas.ipynb contiene listas de las variantes ortográficas de los pueblos que aparecen en censo. También, identificaciones con el nombre actual y otras variables de ayuda para el proceso automático que nos traemos entre manos. Por ejemplo

In [12]:
nombre_extensión = 'parroquia'
parroquias = verifica_matcher(nlp, doc, nombre_extensión, PhraseM=list(ayuda_parroquias.keys()), ayuda=ayuda_parroquias)

Nota: esta función requiere los siguientes módulo o funciones:
 from spacy.matcher import Matcher, PhraseMatcher
 from spacy.tokens import Span
 Si no los usa se producirá un error.
Se han encontrado 2 concordancias con el patrón.


Si deseas ver cómo es este diccionario, descomenta y ejecuta el siguiente código

In [13]:
# parroquias

### Asignamos a cada persona una parroquia
Vamos a crear para ello una extensión para cada entidad denominada `parroquia_asignada`.

In [14]:
Span.set_extension("parroquia_asignada", default=False, force=True)

Ahora calculamos la etiqueta en cada entidad nombrada. Se realizará de forma análoga a como hizimos con la asignación de un año a cada persona.

In [15]:
# Creamos una lista con las claves del diccionario `parroquias`. Y variables auxiliares.
parroquias_claves = list(parroquias.keys())
parroquias_claves.sort()
parroquias_len = len(parroquias_claves)
aux0 = 0
aux1 = 1

# Ahora recorremos las entidades del documento y asignamos el año en cada entidad nombrada
for ent in doc.ents:
    if ent.label_ == "PER":
        # Calculamos el id del token que comienza la entidad:
        comienzo = ent.start
        
        if comienzo < parroquias_claves[0]:
            continue
        else:
            if parroquias_claves[aux0] < comienzo < parroquias_claves[aux1]:
                ent._.parroquia_asignada = parroquias.get(parroquias_claves[aux0]).get('etiqueta')
            elif comienzo > parroquias_claves[aux0] and comienzo > parroquias_claves[aux1]:
                aux0 = aux1
                if aux1 + 1 < parroquias_len:
                    aux1 += 1
                    ent._.parroquia_asignada = parroquias.get(parroquias_claves[aux0]).get('etiqueta')
                else:
                    parroquias_claves.append(comienzo + 10000)
                    aux1 += 1
                    ent._.parroquia_asignada = parroquias.get(parroquias_claves[aux0]).get('etiqueta')

### Veamos algunos ejemplos de lo que hemos conseguido

In [16]:
for ent in doc.ents[20:40]:
    if ent._.parroquia_asignada != None and ent.label_ == 'PER':
        print(ent.text, ent.start , '-----', 'parroquia: ' + str(ent._.parroquia_asignada))

Don Joseph Antonio de Herías 175 ----- parroquia: Herías
Juan García de Zibran 186 ----- parroquia: Herías
don Juan López Castrillón 197 ----- parroquia: Herías
Hijodalgo Notorio 202 ----- parroquia: Herías
Ambrosio de la Rua 212 ----- parroquia: Herías
Juan 219 ----- parroquia: Herías
Nicolás 224 ----- parroquia: Herías
Pedro 226 ----- parroquia: Herías
Ambrosio 230 ----- parroquia: Herías
Hijosdalgo Notorios 232 ----- parroquia: Herías
Pedro Fernández de Mera 240 ----- parroquia: Herías
Lorenzo 247 ----- parroquia: Herías
Juan Fernández Herías 257 ----- parroquia: Herías
Santiago Fernández Loredo 269 ----- parroquia: Herías
Pedro 275 ----- parroquia: Herías


## Obteniendo localidades
En el *notebook* `Z_Ayudas.ipynb` tenemos una lista con todas las entradas de localidades del censo. Algunas de ellas pueden hacer referencia a más de una localidad. En los casos de ambigüedad `verifica_matcher` pregunta al usuario qué escribir. Para la mayoría de casos, la función sugiere la respuesta correcta, ¡pero hay que estar atentos si se estudia el censo al completo y no sólo el ejemplo que está colgado en este repositorio de Github! Véase los siguientes ejemplos:

- "CARCEDO" o "CARZEDO" en el censo puede hacer referencia a "Carcedo de Lago" (los dos primeros), pero es "Carcedo de Lomes" para los dos segundos.
- En "CASTANEDO Y VILLAR DE SAN PEDRO", el PhraseMatcher reconoce bien "CASTANEDO", pero "VILLAR DE SAN PEDRO" tiene tres posibilidades, a saber "Villar de sapos" por reconocer la palabra "VILLAR", "Villar de Castanedo" por reconocer "VILLAR DE SAN PEDRO" y "San Pedro" por reconocer "SAN PEDRO". La correcta es "Villar de Castanedo". Las demás opciones las pasaremos escribiendo la palabra "parar".


IMPORTANTE: Todas las entradas de localidades del documento txt deben estar en mayúsculas y en líneas aisladas. Además, cuando se escribe una etiqueta a mano, es decir, la respuesta no es ni "sí", ni "parar", ni "v", se tiene que escribir igual que lo detalla el diccionario `ayuda_localidades` del *notebook* `Z_Ayudas.ipynb`.

### Aplicamos un patrón de búsqueda y usamos variables de apoyo
De forma general, las variables de apoyo sirven para automatizar la asignación de etiquetas.

In [17]:
nombre_extensión3 = 'lugar'
localidades = verifica_matcher(nlp, doc, nombre_extensión3, PhraseM=list(ayuda_localidades.keys()), ayuda=ayuda_localidades, quitarPM=list(ayuda_parroquias.keys()))

Nota: esta función requiere los siguientes módulo o funciones:
 from spacy.matcher import Matcher, PhraseMatcher
 from spacy.tokens import Span
 Si no los usa se producirá un error.
Se han encontrado 7 concordancias con el patrón.


Si quieres ver cómo es este diccionario, descomenta y ejecuta el siguiente código.

In [18]:
# localidades

Hay entidades de persona que están registrada bajo varias localidades a la vez. Por ejemplo, aquellos bajo la entrada de localidad "COBA, FURADA Y RUBIEIRO". Por ello, a cada persona le asignaremos como lugar una lista de localidades. Para ello necesitamos reestructurar el diccionario `localidades` de esta forma:

In [19]:
localidades_new = {}

for clave, dic in localidades.items():
    if localidades_new.get(dic['id'][0]) == None:
        localidades_new[dic['id'][0]] = {
            'id': dic['id'],
            'etiqueta': [dic['etiqueta']]
        } 
    else: 
        localidades_new[dic['id'][0]]['etiqueta'].append(dic['etiqueta'])
        
        

Si quieres ver cómo es este diccionario, descomenta y ejecuta el siguiente código.

In [20]:
# localidades_new

### Asignamos a cada persona una localidad
Vamos a crear para ello una extensión para cada entidad denominada `lugar_asignado`.

In [21]:
Span.set_extension("lugar_asignado", default=False, force=True)

Ahora calculamos la etiqueta en cada entidad nombrada. Se realizará de forma análoga a como hizimos con la asignación de un año a cada persona.

In [22]:
# Creamos una lista con las claves del diccionario `parroquias`. Y variables auxiliares.
localidades_claves = list(localidades_new.keys())
localidades_claves.sort()
localidades_len = len(localidades_claves)
aux0 = 0
aux1 = 1

# Ahora recorremos las entidades del documento y asignamos la lista de localidades
for ent in doc.ents:
    if ent.label_ == "PER":
        # Calculamos el id del token que comienza la entidad:
        comienzo = ent.start        
        if comienzo < localidades_claves[0]:
            continue
        else:
            if localidades_claves[aux0] < comienzo < localidades_claves[aux1]:
                ent._.lugar_asignado = localidades_new.get(localidades_claves[aux0]).get('etiqueta')
            elif comienzo > localidades_claves[aux0] and comienzo > localidades_claves[aux1]:
                aux0 = aux1
                if aux1 + 1 < localidades_len:
                    aux1 += 1
                    ent._.lugar_asignado = localidades_new.get(localidades_claves[aux0]).get('etiqueta')
                else:
                    localidades_claves.append(comienzo + 10000)
                    aux1 += 1
                    ent._.lugar_asignado = localidades_new.get(localidades_claves[aux0]).get('etiqueta')

## Desasignar lugar a empadronadores locales
Las EENN de personas que aparecen en la sección de "Empadronadores locales" no deben tener un `lugar_asignado`. Sin embargo, de forma automática, por reglas, les ha sido asignado. Debemos revertir esta acción.

In [23]:
nombre_extensión4 = 'empadronadores'
empadronadores = verifica_matcher(nlp, doc, nombre_extensión4, PhraseM=['Empadronadores locales:', 'Empadronadores Locales:'], ayuda={'EMPADRONADORES LOCALES:': 'Emp'})

Nota: esta función requiere los siguientes módulo o funciones:
 from spacy.matcher import Matcher, PhraseMatcher
 from spacy.tokens import Span
 Si no los usa se producirá un error.
Se han encontrado 2 concordancias con el patrón.


Si quieres ver cómo es este diccionario, descomenta y ejecuta el siguiente código.

In [24]:
# empadronadores

### El siguiente paso realiza la desasignación en sí

In [25]:
empadronadores_claves = list(empadronadores.keys())
empadronadores_claves.sort()


registro = []
for i in empadronadores_claves:
    for j in localidades_claves:
        if j >= i:
            registro.append((i, j))
            break

for ent in doc.ents:
    if ent.label_ == 'PER':
        comienzo = ent.start
        for tupla in registro:
            if tupla[0] <= comienzo <= tupla[1]:
                ent._.lugar_asignado = False

## Tabulando los datos
La tabla tendrá las columnas siguientes:
- Nombre
- Año
- Parroquia de
- Longitud parr.
- Latitud parr.
- Localidad 1
- Longitud 1
- Latitud 1
- ...

Hay hasta tres casillas para localidad y su geolocalización porque en el censo completo hay entradas que están agrupados con hasta tres localidades distintas. Por ejemplo, véase en el padrón completo las entradas referentes a "COBA, FURADA Y RUBIEIRO".

In [29]:
# Abrimos el documento para guardar los datos.
with open(direcciónpueblos, 'w', encoding='utf8') as f:
    
    # Escribimos los títulos de tabla
    f.write('Nombre' + '\t' +
            'Año' + '\t' +
            'Parroquia de' + '\t' +
            'Latitud parr.' + '\t' +
            'Longitud parr.' + '\t' +
            'Localidad 1' + '\t' +
            'Latitud 1' + '\t' +
            'Longitud 1' + '\t' +
            'Localidad 2' + '\t' +
            'Latitud 2' + '\t' +
            'Longitud 2' + '\t' + 
            'Localidad 3' + '\t' +
            'Latitud 3' + '\t' +
            'Longitud 3' +     
            '\n')
    
    # Recorremos las entidades nombradas.
    for ent in doc.ents:
        
        if ent.label_ == 'PER':
            Nombre = ent.text
            Año = str(ent._.tiempo)
            Parroquia = ent._.parroquia_asignada
            if Parroquia != False:
                dic = longitud_latitud.get(Parroquia)
                Latitud_parr = str(dic.get('Latitud'))
                Longitud_parr = str(dic.get('Longitud'))
            else:
                Parroquia = str(None)
                Latitud_parr = str(None)
                Longitud_parr = str(None)
            
            localidades = ent._.lugar_asignado
            if localidades == False:
                localidades = []
                
            Localidad1 = str(None)
            Localidad2 = str(None)
            Localidad3 = str(None)
            Latitud1 = str(None)
            Latitud2 = str(None)
            Latitud3 = str(None)
            Longitud1 = str(None)
            Longitud2 = str(None)
            Longitud3 = str(None)
            
            if len(localidades) >= 1:
                Localidad1 = localidades[0]
                info = longitud_latitud.get(Localidad1)
                Longitud1 = str(info['Longitud'])
                Latitud1 = str(info['Latitud'])
                
            if len(localidades) >= 2:
                Localidad2 = localidades[1]
                info = longitud_latitud.get(Localidad2)
                Longitud2 = str(info['Longitud'])
                Latitud2 = str(info['Latitud'])
                
            if len(localidades) >= 3:
                Localidad3 = localidades[2]
                info = longitud_latitud.get(Localidad3)
                Longitud3 = str(info['Longitud'])
                Latitud3 = str(info['Latitud'])
                            
            
            # Escribimos la línea.
            f.write(Nombre + '\t' +
            Año + '\t' +
            Parroquia + '\t' +
            Latitud_parr + '\t' +
            Longitud_parr + '\t' +
            Localidad1 + '\t' +
            Latitud1 + '\t' +
            Longitud1 + '\t' + 
            Localidad2 + '\t' +
            Latitud2 + '\t' +
            Longitud2 + '\t' +
            Localidad3 + '\t' +
            Latitud3 + '\t' +
            Longitud3 +        
            '\n')

## Serialización
Vamos a serializar guardando a parte un diccionario con la información de todas las extensiones definidas en el doc. Al momento de revertir la serialización hay que volver a crearlas.

In [27]:
serializacion(doc, direccionbin_tags)

Nota: esta función requiere los siguientes módulo o funciones:
 import spacy
 from spacy.tokens import DocBin
 Si no los usa se producirá un error.


## Revertir serialización
Ahora, cuando quieras seguir experimentando con los resultados obtenidos, no tienes que ejecutar de nuevo todo el código de cálculo de etiquetas. Basta con que reviertas la serialización con el siguiente código. Nótese que es necesario decirle a SpaCy información de las etiquetas que queremos recuperar.

In [28]:
extensiones = {
    "tiempo": {
        'tipo': 'span',
        'default': False,
        'force': True
    },
    "parroquia": {
        'tipo': 'span',
        'default': False,
        'force': True
    },
    "parroquia_asignada": {
        'tipo': 'span',
        'default': False,
        'force': True
    },
    "lugar_asignado": {
        'tipo': 'span',
        'default': False,
        'force': True
    },
    "lugar": {
        'tipo': 'span',
        'default': False,
        'force': True
    },
    "empadronadores": {
        'tipo': 'span',
        'default': False,
        'force': True
    }
}

doc = revertir_serializacion(direccionbin_tags, nlp, extensiones=extensiones)

Nota: esta función requiere los siguientes módulo o funciones:
 import spacy
 from spacy.tokens import Doc
 from spacy.tokens import DocBin
 Si no los usa se producirá un error.
